In [83]:
import os
import re
import json
import numpy as np
import tensorflow as tf

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import Model, Input
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.python.keras.preprocessing.text import tokenizer_from_json, Tokenizer

In [87]:
class dataset:
    def __init__(self) -> None:
        pass

    # 데이터셋 정보 json 탐색 함수
    def search_json(self, dir):
        paths = []
        count = 0
        for data in os.listdir(dir):
            file_path = os.path.join(dir, data)
            if os.path.isdir(file_path):
                count += self.search_json(file_path)
            elif data.endswith('.json'):
                count += 1
                paths.append(file_path)
        print(paths)
        return count
    
    # 데이터셋 발화자 id 확인 함수
    def id_checker(self, json_path):
        id_sector = []
        with open(json_path, encoding='utf-8-sig') as file:
            json_data = json.load(file) # load json
            for i in range(0, 2):   # id search loop
                id = json_data['dataSet']['dialogs'][i]['speaker']  # id 0,1 indexing
                category = json_data['dataSet']['typeInfo']['category'] # category indexing
                if id and category is not None:
                    id_sector.append(id)
            v1, v2 = id_sector[0], id_sector[1]
        print(f"Server : {v1}\nClient : {v2}")
        print(f"Category : {category}")
        return v1, v2, category

    # def text_preprocesor():
    
    # def integer_indexer():///
    